In [1]:
import qiskit
import cirq
import braket
import numpy as np
from typing import List
from cirq import Circuit as CirqCircuit
from qiskit import QuantumCircuit as QiskitCircuit
from qiskit.quantum_info import Operator as QiskitOperator
from braket.circuits import Circuit as BraketCircuit
from braket.circuits.unitary_calculation import calculate_unitary
from qbraid import circuit_wrapper

### Shared gate circuit functions and unitary helper function

In [2]:
def qiskit_shared_gates_circuit():
    """Returns qiskit `QuantumCircuit` for qBraid `TestSharedGates`."""
    
    qiskit_circuit = QiskitCircuit(4)

    qiskit_circuit.h([0, 1, 2, 3])
    qiskit_circuit.x([0, 1])
    qiskit_circuit.y(2)
    qiskit_circuit.z(3)
    qiskit_circuit.s(0)
    qiskit_circuit.sdg(1)
    qiskit_circuit.t(2)
    qiskit_circuit.tdg(3)
    qiskit_circuit.rx(np.pi/4, 0)
    qiskit_circuit.ry(np.pi/2, 1)
    qiskit_circuit.rz(3*np.pi/4, 2)
    qiskit_circuit.p(np.pi/8, 3)
    qiskit_circuit.sx(0)
    qiskit_circuit.sxdg(1)
    qiskit_circuit.iswap(2, 3)
    qiskit_circuit.swap([0, 1], [2, 3])
    qiskit_circuit.cx(0, 1)
    qiskit_circuit.cp(np.pi/4, 2, 3)

    return qiskit_circuit

In [3]:
def cirq_shared_gates_circuit(rev_qubits=False):
    """Returns cirq `Circuit` for qBraid `TestSharedGates` 
    rev_qubits=True reverses ordering of qubits."""
    
    cirq_circuit = CirqCircuit()
    qubits = [cirq.LineQubit(i) for i in range(4)]
    q3, q2, q1, q0 = qubits if rev_qubits else list(reversed(qubits))
    input_qubit_mapping = {q3:0,q2:1,q1:2,q0:3} if rev_qubits else {q0:0,q1:1,q2:2,q3:3}
        
    cirq_gates = [
        cirq.H(q0),
        cirq.H(q1),
        cirq.H(q2),
        cirq.H(q3),
        cirq.X(q0),
        cirq.X(q1),
        cirq.Y(q2),
        cirq.Z(q3),
        cirq.S(q0),
        cirq.ZPowGate(exponent=-0.5)(q1),
        cirq.T(q2),
        cirq.ZPowGate(exponent=-0.25)(q3),
        cirq.Rx(rads=np.pi/4)(q0),
        cirq.Ry(rads=np.pi/2)(q1),
        cirq.Rz(rads=3*np.pi/4)(q2),
        cirq.ZPowGate(exponent=1/8)(q3),
        cirq.XPowGate(exponent=0.5)(q0),
        cirq.XPowGate(exponent=-0.5)(q1),
        cirq.ISWAP(q2, q3),
        cirq.SWAP(q0, q2),
        cirq.SWAP(q1, q3),
        cirq.CNOT(q0, q1),
        cirq.CZPowGate(exponent=0.25)(q2, q3),
    ]
    
    for gate in cirq_gates:
        cirq_circuit.append(gate)
    
    return cirq_circuit, input_qubit_mapping

In [4]:
def braket_shared_gates_circuit():
    """Returns braket `Circuit` for qBraid `TestSharedGates`."""
    
    braket_circuit = BraketCircuit()

    braket_circuit.h([0, 1, 2, 3])
    braket_circuit.x([0, 1])
    braket_circuit.y(2)
    braket_circuit.z(3)
    braket_circuit.s(0)
    braket_circuit.si(1)
    braket_circuit.t(2)
    braket_circuit.ti(3)
    braket_circuit.rx(0, np.pi / 4)
    braket_circuit.ry(1, np.pi / 2)
    braket_circuit.rz(2, 3 * np.pi / 4)
    braket_circuit.phaseshift(3, np.pi / 8)
    braket_circuit.v(0)
    braket_circuit.vi(1)
    braket_circuit.iswap(2, 3)
    braket_circuit.swap(0, 2)
    braket_circuit.swap(1, 3)
    braket_circuit.cnot(0, 1)
    braket_circuit.cphaseshift(2, 3, np.pi / 4)
    
    return braket_circuit

In [5]:
def to_unitary(circuit):
    """Calculate unitary of a braket, cirq, or qiskit circuit.
    Args:
        circuit (braket, cirq, or qiskit Circuit): The circuit object for which 
            the unitary matrix will be calculated. 
    Returns:
        numpy.ndarray: A numpy array representing the `circuit` as a unitary
    """
    if isinstance(circuit, BraketCircuit):
        return calculate_unitary(circuit.qubit_count, circuit.instructions)
    elif isinstance(circuit, CirqCircuit):
        return circuit.unitary()
    elif isinstance(circuit, QiskitCircuit):
        return QiskitOperator(circuit).data
    else:
        raise TypeError(f"to_unitary calculation not supported for type {type(circuit)}")

### Define circuits and unitaries

In [6]:
braket_circuit = braket_shared_gates_circuit()
cirq_circuit, cirq_input_qubit_mapping = cirq_shared_gates_circuit()
cirq_rev_circuit, _ = cirq_shared_gates_circuit(rev_qubits=True)
qiskit_circuit = qiskit_shared_gates_circuit()

In [7]:
braket_unitary = to_unitary(braket_circuit)
cirq_unitary = to_unitary(cirq_circuit)
cirq_rev_unitary = to_unitary(cirq_rev_circuit)
qiskit_unitary = to_unitary(qiskit_circuit)

In [8]:
qbraid_circuit_braket = circuit_wrapper(braket_circuit)
qbraid_circuit_cirq = circuit_wrapper(cirq_circuit, input_qubit_mapping = cirq_input_qubit_mapping)
qbraid_circuit_qiskit = circuit_wrapper(qiskit_circuit)

In [9]:
def test_transpile(qbraid_circuit, target_package, target_unitary, print_circuit=False):
    transpiled_circuit = qbraid_circuit.transpile(target_package)
    if print_circuit:
        print("transpiled circuit\n", transpiled_circuit)
    transpiled_unitary = to_unitary(transpiled_circuit)
    return np.allclose(transpiled_unitary, target_unitary)

In [10]:
cirq_qiskit_equal = np.allclose(cirq_rev_unitary, qiskit_unitary)
cirq_braket_equal = np.allclose(cirq_rev_unitary, braket_unitary)
qiskit_braket_equal = np.allclose(qiskit_unitary, braket_unitary)

print("all starting unitaries equal")
cirq_qiskit_equal and cirq_braket_equal and qiskit_braket_equal

all starting unitaries equal


True

In [11]:
print("braket to cirq")
# print("original circuit\n", braket_circuit, "\n")
# print("target circuit\n", cirq_circuit, "\n")
test_transpile(qbraid_circuit_braket, "cirq", cirq_unitary)

braket to cirq


True

In [12]:
print("braket to qiskit")
# print("original circuit\n", braket_circuit, "\n")
# print("target circuit\n", qiskit_circuit, "\n")
test_transpile(qbraid_circuit_braket, "qiskit", qiskit_unitary)

braket to qiskit


True

In [13]:
print("qiskit to braket")
# print("original circuit\n", qiskit_circuit, "\n")
# print("target circuit\n", braket_circuit, "\n")
test_transpile(qbraid_circuit_qiskit, "braket", braket_unitary)

qiskit to braket


True

In [14]:
print("qiskit to cirq")
# print("original circuit\n", qiskit_circuit, "\n")
# print("target circuit\n", cirq_circuit, "\n")
test_transpile(qbraid_circuit_qiskit, "cirq", cirq_unitary)

qiskit to cirq


True

In [15]:
print("cirq to braket")
# print("original circuit\n", cirq_circuit, "\n")
# print("target circuit\n", braket_circuit, "\n")
test_transpile(qbraid_circuit_cirq, "braket", braket_unitary)

cirq to braket


True

In [16]:
print("cirq to qiskit")
# print("original circuit\n", cirq_circuit, "\n")
# print("target circuit\n", qiskit_circuit, "\n")
test_transpile(qbraid_circuit_cirq, "qiskit", qiskit_unitary)

cirq to qiskit


True